# Module 12: Manage an Azure Cosmos DB SQL API solution using DevOps practices

- [[Learning path]](https://docs.microsoft.com/en-us/learn/paths/manage-cosmos-db-sql-api-solution-using-devops-practices/?ns-enrollment-type=Collection&ns-enrollment-id=1k8wcz8zooj2nx)
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/30-adjust-throughput-cli-script.html): Adjust provisioned throughput using an Azure CLI script
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/31-create-container-arm-template.html): Create an Azure Cosmos DB SQL API container using Azure Resource Manager templates

## Write management scripts for Azure Cosmos DB SQL API

### Create resources

In this module, we will use Azure CLI to manage Azure Cosmos DB SQL API accounts.

In [ ]:
az cosmosdb create --help

Create the account

In [ ]:
az cosmosdb create `
  --name 'cosmos-dp-420-sql-cli' `
  --resource-group 'rg-dp-420' `
  --default-consistency-level 'eventual' `
  --enable-free-tier 'true' `
  --locations regionName='eastus'

Create a database

In [ ]:
az cosmosdb sql database create `
  --account-name 'cosmos-dp-420-sql-cli' `
  --resource-group 'rg-dp-420' `
  --name 'mydatabase'

Create a container

In [ ]:
az cosmosdb sql container create `
    --account-name 'cosmos-dp-420-sql-cli' `
    --resource-group 'rg-dp-420' `
    --database-name "mydatabase" `
    --name mycontainer `
    --partition-key-path "/my/path" `
    --ttl 1000 `
    --throughput "400"


### Manage index policies

When creating a container, you specify the indexing policy using CLI.

In [ ]:
az cosmosdb sql container create `
    --account-name 'cosmos-dp-420-sql-cli' `
    --resource-group 'rg-dp-420' `
    --database-name "mydatabase" `
    --name mycontainerwithindex `
    --partition-key-path "/my/path" `
    --ttl 1000 `
    --throughput "400" `
    --idx '{\"indexingMode\":\"consistent\",\"automatic\":true,\"includedPaths\":[{\"path\":\"/*\"}],\"excludedPaths\":[{\"path\":\"/headquarters/*\"},{\"path\":\"/\\\"_etag\\\"/?\"}]}’


### Configure database or container-provisioned throughput

You can manage the provisioned throughput for both containers and databases using the CLI.

In [ ]:
az cosmosdb sql database create `
  --account-name 'cosmos-dp-420-sql-cli' `
  --resource-group 'rg-dp-420' `
  --name 'mydatabasewiththroughput' `
  --throughput 400

In [ ]:
az cosmosdb sql database throughput update `
  --account-name 'cosmos-dp-420-sql-cli' `
  --resource-group 'rg-dp-420' `
  --name 'mydatabasewiththroughput' `
  --throughput 600

In [ ]:
az cosmosdb sql container throughput update `
    --account-name 'cosmos-dp-420-sql-cli' `
    --resource-group 'rg-dp-420' `
    --database-name "mydatabase" `
    --name mycontainerwithindex `
    --throughput 1000

### Migrate between standard and autoscale throughput

In [ ]:
az cosmosdb sql container throughput migrate `
    --account-name 'cosmos-dp-420-sql-cli' `
    --resource-group 'rg-dp-420' `
    --database-name "mydatabase" `
    --name mycontainerwithindex `
    --throughput-type 'autoscale'

In [ ]:
az cosmosdb sql container throughput update `
    --account-name 'cosmos-dp-420-sql-cli' `
    --resource-group 'rg-dp-420' `
    --database-name "mydatabase" `
    --name mycontainerwithindex `
    --max-throughput 2000

In [ ]:
az cosmosdb sql container throughput migrate `
    --account-name 'cosmos-dp-420-sql-cli' `
    --resource-group 'rg-dp-420' `
    --database-name "mydatabase" `
    --name mycontainerwithindex `
    --throughput-type 'manual'

### Configure failovers and failover priorities

Add account regions

In [ ]:
az cosmosdb update `
    --name 'cosmos-dp-420-sql-cli' `
    --resource-group 'rg-dp-420' `
    --locations regionName='eastus' failoverPriority=0 isZoneRedundant=False `
    --locations regionName='westus2' failoverPriority=1 isZoneRedundant=False `
    --locations regionName='centralus' failoverPriority=2 isZoneRedundant=False

Remove account regions

In [ ]:
az cosmosdb update `
  --name 'cosmos-dp-420-sql-cli' `
  --resource-group 'rg-dp-420' `
  --locations regionName='eastus' failoverPriority=0 isZoneRedundant=False `
  --locations regionName='westus2' failoverPriority=1 isZoneRedundant=False

Enable automatic failover

In [ ]:
az cosmosdb update `
  --name 'cosmos-dp-420-sql-cli' `
  --resource-group 'rg-dp-420' `
  --enable-automatic-failover 'true'

Change failover priorities

In [ ]:
az cosmosdb failover-priority-change `
  --name 'cosmos-dp-420-sql-cli' `
  --resource-group 'rg-dp-420' `
  --failover-policies 'eastus=0' 'centralus=1' 'westus2=2'

Initiate failovers

In [ ]:
az cosmosdb failover-priority-change `
  --name 'cosmos-dp-420-sql-cli' `
  --resource-group 'rg-dp-420' `
  --failover-policies 'westus2=0' 'eastus=1'

Enable multi-region write

In [ ]:
az cosmosdb update `
  --name 'cosmos-dp-420-sql-cli' `
  --resource-group 'rg-dp-420' `
  --enable-multiple-write-locations 'true'

## Create resource template for Azure Cosmos DB SQL API

### Understand Azure Resource Manager resources

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/create-resource-template-for-azure-cosmos-db-sql-api/media/2-hierarchy.png)

### Author Azure Resource Manager templates

There are three primary resources to define in a specific relationship order when authoring a template for an Azure Cosmos DB SQL API account.

```json
{ 
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentTemplate.json#", 
  "contentVersion": "1.0.0.0", 
  "resources": [ 
      { 
         "type": "Microsoft.DocumentDB/databaseAccounts", 
         "apiVersion": "2021-05-15", 
         "name": "[concat('csmsarm', uniqueString(resourceGroup().id))]",
         "location": "[resourceGroup().location]", 
         "properties": { 
               "databaseAccountOfferType": "Standard", 
               "locations": [ { "locationName": "westus" } ] 
          } 
       }, 
       { 
           "type": "Microsoft.DocumentDB/databaseAccounts/sqlDatabases", 
           "apiVersion": "2021-05-15", 
           "name": "[concat('csmsarm', uniqueString(resourceGroup().id), '/cosmicworks')]", 
           "dependsOn": [ "[resourceId('Microsoft.DocumentDB/databaseAccounts', concat('csmsarm', uniqueString(resourceGroup().id)))]" ], 
           "properties": { "resource": { "id": "cosmicworks" } } 
       }, 
       { 
           "type": "Microsoft.DocumentDB/databaseAccounts/sqlDatabases/containers", 
           "apiVersion": "2021-05-15", 
           "name": "[concat('csmsarm', uniqueString(resourceGroup().id), '/cosmicworks/products')]", 
           "dependsOn": [ "[resourceId('Microsoft.DocumentDB/databaseAccounts', concat('csmsarm', uniqueString(resourceGroup().id)))]",
                          "[resourceId('Microsoft.DocumentDB/databaseAccounts/sqlDatabases', concat('csmsarm', uniqueString(resourceGroup().id)), 'cosmicworks')]" 
                        ], 
           "properties": { "options": { "throughput": 400 }, "resource": { "id": "products", "partitionKey": { "paths": [ "/categoryId" ] } } } 
       } 
   ] 
}

```

### Configure database or container-resources in Bicep

Each template resource uses the same resource type and version between both Azure Resource Manager and Bicep templates.

```bicep
resource Account 'Microsoft.DocumentDB/databaseAccounts@2021-05-15’ = 
{ 
  name: 'csmsbicep${uniqueString(resourceGroup().id)}’ 
  location: resourceGroup().location 
  properties: { 
    databaseAccountOfferType: 'Standard’ 
    locations: [ { locationName: 'westus' } ] 
  } 
}

resource Database 'Microsoft.DocumentDB/databaseAccounts/sqlDatabases@2021-05-15’ = 
{ 
  parent: Account name: 'cosmicworks’ 
  properties: { 
    options: { } 
    resource: { id: 'cosmicworks’ } 
  } 
}

resource Container 'Microsoft.DocumentDB/databaseAccounts/sqlDatabases/containers@2021-05-15’ = 
{ 
  parent: Database name: 'customers’ 
  properties: { 
    resource: { 
      id: 'customers’ 
      partitionKey: { paths: [ '/regionId’ ] } 
    } 
  } 
}
```

### Deploy templates to a resource group

Now that the templates have been defined, use the Azure CLI to deploy either JSON or Bicep Azure Resource Manager templates.


In [ ]:
az deployment group create ` 
  --resource-group rg-dp-420 ` 
  --name 'my-cosmos-db-deployment' `
  --template-file '.\template.json’ `
  --parameters '@.\template.json'

In [ ]:
az deployment group create ` 
  --resource-group rg-dp-420 ` 
  --name 'my-cosmos-db-deployment' `
  --template-file '.\template.bicep 

### Manage index policies – JSON templates

Defining and deploying an indexing policy in JSON templates.

```json
{
    "type": "Microsoft.DocumentDB/databaseAccounts/sqlDatabases/containers",
    "apiVersion": "2021-05-15",
    "name": "[concat('csmsarm', uniqueString(resourceGroup().id), '/cosmicworks/products')]",
    "dependsOn": [
        "[resourceId('Microsoft.DocumentDB/databaseAccounts', concat('csmsarm', uniqueString(resourceGroup().id)))]",
        "[resourceId('Microsoft.DocumentDB/databaseAccounts/sqlDatabases', concat('csmsarm', uniqueString(resourceGroup().id)), 'cosmicworks')]"
    ],
    "properties": {
        "options": {
            "throughput": 400
        },
        "resource": {
            "id": "products",
            "partitionKey": {
                "paths": [
                    "/categoryId"
                ]
            },
            "indexingPolicy": {
                "indexingMode": "consistent",
                "automatic": true,
                "includedPaths": [
                    {
                        "path": "/price/*"
                    }
                ],
                "excludedPaths": [
                    {
                        "path": "/*"
                    }
                ]
            }
        }
    }
}
```

### Manage index policies – Bicep templates

```bicep
resource Container 'Microsoft.DocumentDB/databaseAccounts/sqlDatabases/containers@2021-05-15' = { 
    parent: Database 
    name: 'customers’ 
    properties: { 
        resource: { 
            id: 'customers’ 
            partitionKey: { paths: [ '/regionId' ] } 
            indexingPolicy: { 
                indexingMode: 'consistent’ 
                automatic: true includedPaths: [ { path: '/address/*' } ] 
                excludedPaths: [ { path: '/*' } ] 
            } 
        } 
    } 
}
```

## Demo teardown

In [ ]:
az cosmosdb delete --name cosmos-dp-420-sql-cli --resource-group rg-dp-420 --yes